In [1]:
!pip install setuptools==58.0.0 # 降级setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.3/816.3 kB 50.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 72.1.0
    Uninstalling setuptools-72.1.0:
      Successfully uninstalled setuptools-72.1.0


In [ ]:
!sh environment_setup.sh

##### 加载尝试

In [ ]:
# 剪枝后size不匹配问题：
# 1. strict=False,不行
# 2. 重新加载

In [7]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, LlamaForCausalLM
from safetensors.torch import load_file


model_name = "../outputs/vila-llm-pruned-mask"

# # 使用自定义的 Llama 模型
model = AutoModelForCausalLM.from_pretrained(
    "../outputs/vila-llm-pruned-mask",
    torch_dtype=torch.bfloat16,
    ignore_mismatched_sizes=True,
    use_safetensors=True
).to("cuda")


# config = AutoConfig.from_pretrained("../outputs/VILA1.5-3b/llm")
# config.hidden_size = 2432  # 根据剪枝后的模型调整hidden_size
# config.num_attention_heads = 19  # 根据剪枝后的模型调整hidden_size

# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16,ignore_mismatched_sizes=True,config=config,
#     use_safetensors=True).to("cuda")

# state_dict = load_file("../outputs/VILA1.5-3b/llm/model-00001-of-00002.safetensors")
# model.load_state_dict(state_dict, strict=False)

# # 加载第二部分权重
# state_dict_part2 = load_file("../outputs/VILA1.5-3b/llm/model-00002-of-00002.safetensors")
# model.load_state_dict(state_dict_part2, strict=False)

tokenizer = AutoTokenizer.from_pretrained('../outputs/VILA1.5-3b/llm')
input_text = "Tell me something about large language models:"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=100, num_return_sequences=1, do_sample=True)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tell me something about large language models: what is happening at Google’s artificially intelligent language and context-based. How do you explain why these giant corporations are getting in on trend a little late? The company’s artificial intelligence programs are an interesting example of a technology that may be used to the benefit of the technology.


In [ ]:
# 需要保证整除

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, LlamaForCausalLM
from safetensors.torch import load_file

class CustomLlamaForCausalLM(LlamaForCausalLM):
    def __init__(self, config):
        print(config)
        config.num_attention_heads = 16
        config.head_dim = 128
        super().__init__(config)
        
        for layer in self.model.layers:
            layer.self_attn.q_proj = nn.Linear(2560, 2048)  # 自定义尺寸
            layer.self_attn.k_proj = nn.Linear(2560, 2048)   # 自定义尺寸
            layer.self_attn.v_proj = nn.Linear(2560, 2048)   # 自定义尺寸
            layer.self_attn.o_proj = nn.Linear(2048, 2560)  # 自定义尺寸

# # 使用自定义的 Llama 模型
model = CustomLlamaForCausalLM.from_pretrained(
    "../outputs/VILA1.5-3b/llm",
    ignore_mismatched_sizes=True,
    use_safetensors=True
).to("cuda")
model.num_attention_heads = 16
model.head_dim = 128

model_name = '../outputs/VILA1.5-3b/llm'
# config = AutoConfig.from_pretrained("../outputs/VILA1.5-3b/llm")
# config.hidden_size = 2432  # 根据剪枝后的模型调整hidden_size
# config.num_attention_heads = 19  # 根据剪枝后的模型调整hidden_size

# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16,ignore_mismatched_sizes=True,config=config,
#     use_safetensors=True).to("cuda")

# state_dict = load_file("../outputs/VILA1.5-3b/llm/model-00001-of-00002.safetensors")
# model.load_state_dict(state_dict, strict=False)

# # 加载第二部分权重
# state_dict_part2 = load_file("../outputs/VILA1.5-3b/llm/model-00002-of-00002.safetensors")
# model.load_state_dict(state_dict_part2, strict=False)

tokenizer = AutoTokenizer.from_pretrained(model_name)
input_text = "Tell me something about large language models:"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=100, num_return_sequences=1, do_sample=True)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
from transformers import AutoModelForCausalLM
from safetensors.torch import load_file
import torch

# 1. 加载原始预训练模型
original_model_path = "../../../base_model/VILA1.5-3b/llm"
model = AutoModelForCausalLM.from_pretrained(original_model_path)

# 2. 加载新的 safetensors 权重文件
safetensor_path = "../outputs/VILA1.5-3b/llm/model-00001-of-00002.safetensors"
new_weights = load_file(safetensor_path)

# 3. 替换指定模块的权重（比如 self_attn.q_proj）
with torch.no_grad():  # 避免自动梯度跟踪
    for name, param in model.named_parameters():
        if name in new_weights:
            print(f"Replacing weights for {name}")
            model.named_parameters()[name] = new_weights

# 4. 检查替换是否成功
for name, param in model.named_parameters():
    if name in new_weights:
        assert torch.allclose(param, new_weights[name]), f"Weight replacement for {name} failed!"

# 5. 保存替换后的模型
output_path = "../path_to_save_new_model"
model.save_pretrained(output_path)

### 启动训练

In [3]:
!pip install wandb
import wandb
wandb.login(key='8b0b2edc05fa3d04079967e01434d12b52f5312e')

True

In [7]:
!ls /home/ec2-user/SageMaker/efs/anker_llm_compression/scripts/outputs/VILA1.5-3b/llm

config.json			  special_tokens_map.json
generation_config.json		  tokenizer_config.json
model-00001-of-00002.safetensors  tokenizer.model
model-00002-of-00002.safetensors  vila-llm-pruned
model.safetensors.index.json


In [ ]:
# 获取当前llama代码

In [67]:
!cp /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/llama/modeling_llama.py modeling_llama_origin.py

In [ ]:
# 用修改后的代码覆盖原有代码

In [37]:
!cp modeling_llama_modified.py /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/llama/modeling_llama.py

In [ ]:
# 检查配置文件

In [ ]:
!cat /home/ec2-user/SageMaker/efs/anker_llm_compression/scripts/outputs/VILA1.5-3b-normal/llm/config.json

In [30]:
!cp config_chenwei.json ../outputs/VILA1.5-3b-normal/llm/config.json

In [ ]:
!cat ../outputs/VILA1.5-3b-normal/llm/config.json

In [ ]:
!bash scripts/v1_5/release/8b/3_sft_anker_a100.sh /home/ec2-user/SageMaker/efs/anker_llm_compression/scripts/outputs/VILA1.5-3b-normal/ VILA1.5-3b-masked-chenwei-test >> training_log.txt 

scripts/v1_5/release/8b/3_sft_anker_a100.sh: line 3: scontrol: command not found
scripts/v1_5/release/8b/3_sft_anker_a100.sh: line 6: scontrol: command not found
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Did not find AutoResume SDK!
Did not find AutoResume SDK!
Did not find AutoResume SDK!
Did not find AutoResume SDK!
Did not find AutoResume SDK!
Did not find AutoResume SDK!
Did not find AutoResume SDK!
Did not find AutoResume SDK!
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it 

In [ ]:
!cat checkpoints/VILA1.5-3b-masked-first4-v1/llm/config.json

### 评测

In [22]:
!bash scripts/v1_5/eval/anker.sh checkpoints/VILA1.5-3b-masked-chenwei-v1 VILA1.5-3b-anker-mask-chenwei-v2 >> test_log.txt 
# !python scripts/v1_5/eval/analyzeResults_VHU_v6.py  --result-file ./result/VILA1.5-3b-anker-mask-chenwei-v1.txt --fixed-precision 0.95

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.45s/it]
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [16]:
!python scripts/v1_5/eval/analyzeResults_VHU_v6.py  --result-file ./result/VILA1.5-3b-anker-mask-chenwei-v1.txt --fixed-precision 0.95

TP: 159, TN: 160, FP: 40, FN: 41, Total: 400
Accuracy: 79.75%
Precision: 79.90%
Recall: 79.50%
F1_score: 79.70%

Fixed Precision: 95.00%, threshold: 0.821962833404541
True Precision:94.64%, Recall: 53.00%, FPPI: 3.00%

Fixed Recall: 95.00%, threshold: 0.1563628911972046
True Recall: 95.00%, Precision: 62.71%, FPPI: 56.50%

Fixed FPPI: 2.00%, threshold: 0.8264954090118408
True FPPI: 2.00%, Precision: 96.30%, Recall: 52.00%

FPPI: 20.00% at threshold 0.5
FPPI: 16.00% at threshold 0.6
FPPI: 4.00% at threshold 0.8
FPPI: 0.00% at threshold 0.9
FPPI: 10.50% at threshold 0.7
FPPI: 0.00% at threshold 0.95



In [23]:
!python scripts/v1_5/eval/analyzeResults_VHU_v6.py  --result-file ./result/VILA1.5-3b-anker-mask-first4-v1.txt --fixed-precision 0.95

TP: 159, TN: 160, FP: 40, FN: 41, Total: 400
Accuracy: 79.75%
Precision: 79.90%
Recall: 79.50%
F1_score: 79.70%

Fixed Precision: 95.00%, threshold: 0.8173599243164062
True Precision:95.00%, Recall: 57.00%, FPPI: 3.00%

Fixed Recall: 95.00%, threshold: 0.16476714611053467
True Recall: 95.00%, Precision: 65.74%, FPPI: 49.50%

Fixed FPPI: 2.00%, threshold: 0.8352636694908142
True FPPI: 2.00%, Precision: 96.26%, Recall: 51.50%

FPPI: 20.50% at threshold 0.5
FPPI: 16.50% at threshold 0.6
FPPI: 4.50% at threshold 0.8
FPPI: 0.00% at threshold 0.9
FPPI: 11.00% at threshold 0.7
FPPI: 0.00% at threshold 0.95

